In [12]:
import numpy as np

def sigmoid(Z):
    """
    Implements the sigmoid activation in numpy
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    cache -- returns Z as well, useful during backpropagation
    """
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):
    """
    Implement the RELU function.
    Arguments:
    Z -- Output of the linear layer, of any shape
    Returns:
    A -- Post-activation parameter, of the same shape as Z
    cache -- a python dictionary containing "A" ; stored for computing the backward pass efficiently
    """
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache


def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.
    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently
    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
#     print(Z)
#     print(dZ)
    # When z <= 0, you should set dz to 0 as well.
    #print(Z<=0)   # output is boolean
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.
    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently
    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(1)
    
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters =  { "W1": W1,
                    "b1": b1,
                    "W2": W2,
                    "b2": b2 }
    
    return parameters

In [ ]:
def initialize_parameters_deep(layer_dims):

    np.random.seed(3)
    parameters = {} # it is a dictionary with ['W+str(l)'],['b+str(l)'] as keys and the randomly generated metrices as their values
    L = len(layer_dims)
#     print("initialised parameters:")
    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * 0.02
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
#         print('W' + str(l)+":"+str(parameters['W' + str(l)]))
#         print('b' + str(l)+"shape :"+str((parameters['b' + str(l)]).shape))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l],layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
        
       
        
    return parameters

In [ ]:
def linear_forward(A, W, b):
    
#     print("W shape"+str(W.shape))
#     print("Ashape"+str(A.shape))
#     print("b shape"+str(b.shape))
    Z = np.dot(W,A)+b
#     print("Z shape:"+str(Z.shape))
#     print("Z="+str()+": "+str(Z))
        
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)  # Z
        A, activation_cache = sigmoid(Z)                # a
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [ ]:
def L_model_forward(X, parameters):
    
    caches = []
    A = X
    L = len(parameters) // 2
    
    for l in range(1, L):
        
        A_prev = A
#         print("A_prev values for layer "+str(l)+":"+str(A))
#         print("A_prev shape:"+str(l)+str(A_prev.shape))
#         print("Z"+str(l)+"="+str('W' +str(l))+" x "+str('b' + str(l)))
        A, cache = linear_activation_forward(A_prev, parameters['W' +str(l)], parameters['b' + str(l)], activation ="relu") #relu
#         print("Z"+str(l)+"="+str('W' +str(l))+" x "+str('b' + str(l)))
        caches.append(cache)
#         print("A shape:"+str(l)+str(A.shape))
#         print("A values for layer "+str(l)+":"+str(A))
    
#     print("Z="+str('W' +str(L))+" x "+str('b' + str(L)))
    Al, cache = linear_activation_forward(A, parameters['W' + str(L)],parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
#     print("Final layer Output:"+str(Al))
    
        
    assert(Al.shape == (1,X.shape[1]))
    
    return Al, caches

In [ ]:
def compute_cost(Al, Y):
    
    m = Y.shape[1]
    
    cost = -(np.dot(Y,np.log(Al.T))+np.dot(1-Y,np.log(1-Al).T))/m
    
    cost = np.squeeze(cost)
    
    assert(cost.shape == ())
    
    return cost

In [1]:
def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [2]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
#         print("dZ: "+str(dZ))
#         print("dA_prev: "+str(dA_prev))
#         print("dW: "+str(dW))
#         print("db: "+str(db))
        
    elif activation == "sigmoid":
        
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
#         print("dZ: "+str(dZ))
        
#         print("dA_prev: "+str(dA_prev))
#         print("dW: "+str(dW))
#         print("db: "+str(db))
    
    return dA_prev, dW, db

In [3]:
#new cpoy of L_model_backward
def L_model_backward(Al, Y, caches):
    
    grads = {}
    L = len(caches)
#     print("caches length: "+str(L))
    m = Al.shape[1] 
    Y = Y.reshape(Al.shape)
    
    
    
    dAl = - (np.divide(Y, Al) - np.divide(1 - Y, 1 - Al))
    grads["dA"+str(L)] = dAl
#     print("dAl: "+str(dAl))
    
    current_cache = caches[L-1]
    
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAl, current_cache, "sigmoid")
    
    for l in reversed(range(L-1)):
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, "relu") #relu
        grads["dA" + str(l)] = dA_prev_temp
#         print("dA" + str(l)+" :" +str(grads["dA" + str(l)]))
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
        
    return grads

In [4]:
def update_parameters(parameters, grads, alpha):
    
    L = len(parameters) // 2

    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - alpha * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - alpha * grads["db" + str(l + 1)]
#         print("W"+str(l+1)+": "+str(parameters["W" + str(l+1)]))
#         print("b"+str(l+1)+": "+str(parameters["b" + str(l+1)]))
        
#     print("W4:"+str(parameters["W4"]))
#     print("b4:"+str(parameters["b4"]))
    
    return parameters

In [5]:
def load_data():
    with h5py.File('train_catvnoncat.h5', "r") as train_dataset:
        train_set_x_orig = np.array(train_dataset["train_set_x"][:])
        train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    with h5py.File('test_catvnoncat.h5', "r") as test_dataset:
        test_set_x_orig = np.array(test_dataset["test_set_x"][:])
        test_set_y_orig = np.array(test_dataset["test_set_y"][:])
        classes = np.array(test_dataset["list_classes"][:])

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [6]:
def image2vec(train_x,test_x):
    
    train_x_vec = train_x.reshape(train_x_orig.shape[0], -1).T
    test_x_vec = test_x.reshape(test_x_orig.shape[0], -1).T
    
    train_X = train_x_vec/255
    test_X = test_x_vec/255
    
    return train_X,test_X

In [10]:
def L_layer_model(X, Y, layers_dims, alpha = 0.0075, num_iterations = 300, print_cost=False):
    
    np.random.seed(1)
    costs = []
    
    parameters = initialize_parameters_deep(layers_dims)
#     print("Initialised parameter:")
   # for k in range(len(parameters)//2):
            
#             print("W{} : {}".format(k+1,parameters["W"+str(k+1)]))
#             print("b{} : {}".format(k+1,parameters["b"+str(k+1)]))
    
    for i in range(0, num_iterations):
        
        AL, caches = L_model_forward(X, parameters)
       # print("AL values of current layer:"+str(AL))
        #print(str(i)+"th iteration")
        
        cost = compute_cost(AL, Y)
       # print("cost after {}th iteration: {}".format(i, cost))
        grads = L_model_backward(AL, Y, caches)
        
        parameters = update_parameters(parameters, grads, alpha)
        #for j in range(len(parameters)//2):
            
#             print("W{} for {}th iteraton : {}".format(j+1,i,parameters["W"+str(j+1)]))
#             print("b{} for {}th iteraton : {}".format(j+1,i,parameters["b"+str(j+1)]))
#         print("W2 for {}th iteraton : {}".format(i,parameters["W2"]))
#         print("b2 for {}th iteraton : {}".format(i,parameters["b2"]))
        
        
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
        
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(alpha))
    plt.show()
    
    return parameters